In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import project_env as pe
%matplotlib inline

In [2]:
#bring in BOFI_NBR, SCREENING_DISP_CODE
data_simple = pd.read_csv('data_simple.csv', encoding = "ISO-8859-1", low_memory=False)
data_simple = data_simple[['SCREENING_DISP_CODE','UNIQUE_ID','BOFI_NBR']]

In [3]:
bin_features = pd.read_csv('df_bin_features.csv', encoding = "ISO-8859-1", low_memory=False)
num_features = pd.read_csv('df_num_features.csv', encoding = "ISO-8859-1", low_memory=False)
date_features = pd.read_csv('df_date_features.csv', encoding = "ISO-8859-1", low_memory=False)
cat_features = pd.read_csv('df_cat_features.csv', encoding = "ISO-8859-1", low_memory=False)
rearrest = pd.read_csv('df_rearrest_times.csv', encoding = "ISO-8859-1", low_memory=False)

print('bin_features: %s' %(str(rearrest.shape)))
print('num_features: %s' %(str(num_features.shape)))
print('date_features: %s' %(str(date_features.shape)))
print('cat_features: %s' %(str(cat_features.shape)))
print('rearrest: %s' %(str(rearrest.shape)))

bin_features: (269543, 2)
num_features: (280294, 5)
date_features: (272088, 16)
cat_features: (280294, 11)
rearrest: (269543, 2)


In [4]:
merged = pd.merge(rearrest, \
                 bin_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 num_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 date_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 cat_features,\
                 on='UNIQUE_ID', \
                 how='left')
merged = pd.merge(merged, \
                 data_simple,\
                 on='UNIQUE_ID', \
                 how='left')

In [5]:
#identify arrests where at least one charge was accepted
accepted = merged[merged['SCREENING_DISP_CODE']==230][['BOFI_NBR','ARREST_DATE']]

#drop rows where at least one charge was accepted during that arrest
data_not_charged = pd.merge(merged, \
                 accepted, \
                 on=['BOFI_NBR','ARREST_DATE'], \
                 how='outer',\
                 indicator = True)
data_not_charged = data_not_charged[data_not_charged['_merge']=='left_only'].drop('_merge', axis=1)

#remove duplicate arrests on same day
data_not_charged = data_not_charged[data_not_charged['NEXT_ARREST_TIME']!='Delete']

#convert NEXT_ARREST_TIME to numeric
data_not_charged['NEXT_ARREST_TIME'] = data_not_charged['NEXT_ARREST_TIME'].apply(pd.to_numeric)

In [7]:
data_test, data_train, data_val = pe.split_data(data_not_charged, test_split=.2,  \
                                                train_split=.64, by_var='ARREST_DATE_y', random_state=1)

Test Data 25068
(100272, 40)
Val Data 20055
Train Data 80217


In [19]:
data_train.groupby(['ARREST_DATE_y'])['ARREST_DATE_y'].count()

ARREST_DATE_y
1988.0    1678
1989.0    7783
1990.0    9769
1991.0    7280
1992.0    6598
1993.0    4770
1994.0    6093
1995.0    5106
1996.0    5929
1997.0    8450
1998.0    9432
1999.0    7329
Name: ARREST_DATE_y, dtype: int64

In [16]:
data_val.groupby(['ARREST_DATE_y'])['ARREST_DATE_y'].count()

ARREST_DATE_y
1988.0     420
1989.0    1946
1990.0    2443
1991.0    1820
1992.0    1649
1993.0    1193
1994.0    1523
1995.0    1277
1996.0    1482
1997.0    2112
1998.0    2358
1999.0    1832
Name: ARREST_DATE_y, dtype: int64

In [17]:
data_train.groupby(['ARREST_DATE_y'])['ARREST_DATE_y'].count()

ARREST_DATE_y
1988.0    1678
1989.0    7783
1990.0    9769
1991.0    7280
1992.0    6598
1993.0    4770
1994.0    6093
1995.0    5106
1996.0    5929
1997.0    8450
1998.0    9432
1999.0    7329
Name: ARREST_DATE_y, dtype: int64